**Show all your work for full credit. Each source code you submit should include detailed comments and instructions on how to run it in order to confirm that it works as expected. If the program that does not run or throws runtime errors, it cannot be graded. You can refer to the programming guidelines from the TAs here: https://tinyurl.com/CPEG-472-672-Programming-Guide/**

**This is an individual assignment and each student should work on their own. Ensure you don't share any code online or with others (note, using Replit, GitHub and similar online platforms can make your code accessible to others).**

**To submit the assignment, you need to use Jupyter Notebook with the provided cell blocks and follow the naming conventions and instructions posted here: https://tinyurl.com/CPEG-472-672-Programming-Guide/**

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and section below:

In [17]:
NAME = "Shruthilaya Arun"
#SECTION = "472"
SECTION = "672"

---

## Extra Credit: Optional Question 5 [35 extra points]

### Implement a timing attack to a MAC that uses byte-by-byte verification taking variable time. Modify Listing 7-1 to add a measurable delay before each by comparison. Then implement two programs P1 and P2 that communicate as follows: P1 creates a forged message/tag pair and sends it to P2 while measuring the response time. P2 knows the MAC secret key and performs byte-by-byte verification that is vulnerable to a timing attack. If the tag verification fails, P2 returns an error immediately. Your P1 program should be able to recover a valid tag for any random message using the timing vulnerability of P2.

In [26]:
# YOUR CODE HERE
import hmac
import hashlib
import time
SECRET_KEY = b'secret_key'
def create_mac(message):
    return hmac.new(SECRET_KEY,message,hashlib.sha1).digest() # create mac
#listing 7-1
def compare_mac(x, y, n, delay=0.05):
    for i in range(n):
        if x[i] != y[i]:
            return False
        time.sleep(delay)  # add delay
    return True
def verify_mac(message,tag):
    expected_tag=create_mac(message)
    return compare_mac(tag,expected_tag,len(expected_tag)) # verify mac
#server response
def server_verify(message,tag):
    if verify_mac(message,tag):
        return "Valid MAC"
    else:
        return "Invalid MAC"

In [27]:
message = b"attack"
correct_tag = create_mac(message)
print(f"Correct Tag: {correct_tag.hex()}")
print(server_verify(message, correct_tag))  


Correct Tag: 4b3841002abaf742b4b1ff34fb1b7887f74a8eb4
Valid MAC


In [28]:
import time
def send_to_server(message,tag):
    start_time=time.time()
    response=server_verify(message,tag) # call server
    end_time=time.time()
    time_taken=end_time-start_time # calculate time taken
    return response,time_taken
def forge_mac(message):
    forged_tag=bytearray(32) #byte array for forged mac
    for i in range(32):
        best_time=0 # keep track of  longest response time
        best_byte=0 # keep track of byte with longest response time
        for byte_guess in range(256):
            forged_tag[i]=byte_guess 
            response,response_time=send_to_server(message,forged_tag) # send forged mac
            if response_time>best_time: 
                best_time=response_time # update response time
                best_byte=byte_guess # set as best guess
        forged_tag[i]=best_byte
    return bytes(forged_tag)


In [29]:
message = b"attack"
forged_mac = forge_mac(message)
print(f"Forged MAC: {forged_mac.hex()}")
print(server_verify(message, forged_mac))  # Verify forged MAC


Forged MAC: 4b639fb66e57ddbda28eaa88472e16a467ef4eaa7f1e83983a0733f91adb0dcc
Invalid MAC
